##CV features

In [1]:
!pip install advertools

In [2]:
!pip install ntlk

ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [3]:
import re
import logging 
import gensim
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
import xgboost as xgb 
from time import time 
import pandas as pd
import time
import nltk
from nltk.corpus import stopwords
# Config
import advertools as adv
from tqdm import tqdm 
from utils import metrics
from functools import reduce
from utils import featureScore
from utils import folderPath
from textblob import TextBlob
from sklearn.manifold import TSNE
from collections import defaultdict
from gensim.models import FastText
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from typing import List, Callable, Dict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from utils import hyperparameterTuning_RandomForest
from utils import hyperparameterTuning_XGBoost
from utils import hyperparameterTuning_MLP
from utils import plot_feature_importance
from sklearn.model_selection import train_test_split
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [4]:
df = pd.read_csv('/content/drive/MyDrive/NLP/CvFeatures.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'image_type', 'account', 'caption',
       'profile_name', 'profile_image_link', 'biography', 'id', 'external_url',
       'following', 'likes', 'media_type', 'posts_count', 'followers',
       'is_verified', 'datetime', 'image_url.1', 'url', 'comments',
       'input_account', 'input_posts_number', 'input_date_of_post',
       'input_post_type', 'input_stop_id', 'warning', 'error', 'tags',
       'confidence_score', 'accent_color', 'is_bw', 'dominant_colors',
       'bg_color', 'fore_color', 'industry', '_category', 'region'],
      dtype='object')

In [6]:
df

,Unnamed: 0,Unnamed: 0.1,image_type,account,caption,profile_name,profile_image_link,biography,id,external_url,...,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry,_category,region
0,0,0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.620000e+18,http://www.yeastbakery.com/,...,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",hospitality,bakery,UK
1,1,1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.790000e+18,http://www.yeastbakery.com/,...,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",hospitality,bakery,UK
2,2,2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.690000e+18,http://www.yeastbakery.com/,...,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality,bakery,UK
3,3,3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.670000e+18,http://www.yeastbakery.com/,...,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality,bakery,UK
4,4,4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,https://instagram.fman2-1.fna.fbcdn.net/v/t51....,We are now open at our new site: Unit 1 Canal ...,2.620000e+18,http://www.yeastbakery.com/,...,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",hospitality,bakery,UK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,88511,88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,https://scontent-lcy1-1.cdninstagram.com/v/t51...,"Local Family Fishmongers & Butchers , fresh fi...",2.700000e+18,http://sandysfish.net/,...,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501...",retail,fishmongers / butchers,UK
88512,88512,88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,https://scontent-lcy1-1.cdninstagram.com/v/t51...,"Local Family Fishmongers & Butchers , fresh fi...",2.700000e+18,http://sandysfish.ne

In [7]:
df.drop(df.columns.difference(['account','profile_name','biography','image_type','caption','comments','tags','following','confidence_score','accent_color','is_bw','dominant_colors','bg_color','fore_color','posts_count','followers','tags','confidence_score','accent_color','is_bw','dominant_colors','bg_color','fore_color','industry','datetime','likes']), 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [8]:
df

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,datetime,comments,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,1.626600e+12,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",hospitality
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,1.646920e+12,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",hospitality
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,1.634730e+12,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,1.633090e+12,3.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,1.626930e+12,6.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",hospitality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,1.636550e+12,0.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501...",retail
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,1.636550e+12,0.0,"['text', 'poster', 'cartoon']","[0.9999863505363464, 0.9184243679046631, 0.910...","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",retail
88513,"['Not Clip', 'Not LineDraw']",sandysfishmongers,"A unique amber coloured blue veined cheese , w...",Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,11.0,1863.0,2321.0,1.636540e+12,2.0,"['text', 'yellow', 'packaging and labeling', '...","[0.9997624158859253, 0.9444

In [9]:
data = df[df['account'].notna()]
data

,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,datetime,comments,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,1.626600e+12,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",hospitality
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,1.646920e+12,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",hospitality
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,1.634730e+12,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,1.633090e+12,3.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,1.626930e+12,6.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",hospitality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.636710e+12,1.0,"['text', 'book', 'screenshot', 'brochure', 'pr...","[0.9999996423721313, 0.9184167385101318, 0.910...","[0.011764705882352941, 0.5019607843137255, 0.7...",0,"[1.0, 1.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 1.0]",retail
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,1.636620e+12,0.0,"['building', 'text', 'electronic signage', 'ch...","[0.9849988222122192, 0.9520490169525146, 0.921...","[0.6470588235294118, 0.1568627450980392, 0.149...",0,"[0.5019607843137255, 0.5019607843137255, 0.501...","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",retail
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,1.636550e+12,0.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.0078431

In [10]:
data.to_csv('CV.csv')

In [11]:
df=pd.read_csv('/content/drive/MyDrive/NLP/CV.csv')

In [12]:
df

,Unnamed: 0,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,datetime,comments,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color,industry
0,0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,1.626600e+12,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",hospitality
1,1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,1.646920e+12,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",hospitality
2,2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,1.634730e+12,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
3,3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,1.633090e+12,3.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",hospitality
4,4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,1.626930e+12,6.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",hospitality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,1.636710e+12,1.0,"['text', 'book', 'screenshot', 'brochure', 'pr...","[0.9999996423721313, 0.9184167385101318, 0.910...","[0.011764705882352941, 0.5019607843137255, 0.7...",0,"[1.0, 1.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 1.0]",retail
88510,88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,1.636620e+12,0.0,"['building', 'text', 'electronic signage', 'ch...","[0.9849988222122192, 0.9520490169525146, 0.921...","[0.6470588235294118, 0.1568627450980392, 0.149...",0,"[0.5019607843137255, 0.5019607843137255, 0.501...","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",retail
88511,88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,1.636550e+12,0.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073

In [13]:
df['caption_length'] = df['caption'].str.len()
df['biography_length'] = df['biography'].str.len()
# df['caption_upper'] = df['caption'].str.findall(r'[A-Z]').str.len()
df['po_co'] = df['posts_count']/(df['comments']+1)
df['pof'] = df['posts_count']/(df['following']+1)
df['user_count'] = df.groupby('account')['posts_count'].transform('count')
df['profile_name_len'] = df['profile_name'].str.len()
df['fol'] = df['followers']/(df['following']+1)
df['act'] = df['comments']/(df['followers']+1)
df['pos'] = df['posts_count']/(df['followers']+1)
df['comments_max'] = df.groupby('account')['comments'].transform('max')
df['comments_min'] = df.groupby('account')['comments'].transform('min')
df['comments_mean'] = df.groupby('account')['comments'].transform('mean')
df['comments_std'] = df.groupby('account')['comments'].transform('std')
df['comments_followers']=df['comments']*df['followers']
df['followers_comments_mean']=df['comments_mean']*df['followers']
df['fol2']=df['fol']*df['fol']
df['fol_pos']=df['fol']*df['pos']
df['fol_pow'] = df['followers']*(df['following'])
df['po_co_pow'] = df['posts_count']*(df['comments'])
df['comments_mean_diff'] = df['comments'] / (df['comments_mean']+1)

In [14]:
df

,Unnamed: 0,image_type,account,caption,profile_name,biography,following,likes,posts_count,followers,...,comments_min,comments_mean,comments_std,comments_followers,followers_comments_mean,fol2,fol_pos,fol_pow,po_co_pow,comments_mean_diff
0,0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,29.0,280.0,8702.0,...,0.0,5.85000,8.949624,0.0,50906.700000,607.699315,0.793110,3063104.0,0.0,0.000000
1,1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,44.0,280.0,8702.0,...,0.0,5.85000,8.949624,0.0,50906.700000,607.699315,0.793110,3063104.0,0.0,0.000000
2,2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,209.0,280.0,8702.0,...,0.0,5.85000,8.949624,78318.0,50906.700000,607.699315,0.793110,3063104.0,2520.0,1.313869
3,3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,95.0,280.0,8702.0,...,0.0,5.85000,8.949624,26106.0,50906.700000,607.699315,0.793110,3063104.0,840.0,0.437956
4,4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,Yeast Bakery,We are now open at our new site: Unit 1 Canal ...,352.0,101.0,280.0,8702.0,...,0.0,5.85000,8.949624,52212.0,50906.700000,607.699315,0.793110,3063104.0,1680.0,0.875912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,10.0,1863.0,2321.0,...,0.0,0.81982,1.063427,2321.0,1902.801802,16.013796,3.210686,1343859.0,1863.0,0.549505
88510,88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,50.0,1863.0,2321.0,...,0.0,0.81982,1.063427,0.0,1902.801802,16.013796,3.210686,1343859.0,0.0,0.000000
88511,88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,8.0,1863.0,2321.0,...,0.0,0.81982,1.063427,0.0,1902.801802,16.013796,3.210686,1343859.0,0.0,0.000000
88512,88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,Fishmongers/ButchersTwickenham,"Local Family Fishmongers & Butchers , fresh fi...",579.0,9.0,1863.0,2321.0,...,0.0,0.81982,1.063427,0.0,1902.801802,16.013796,3.210686,1343859.0,0.0,0.000000


In [15]:
len(df.account.unique())

981

In [16]:
df.drop(['Unnamed: 0'], axis = 1,inplace=True)
# df.drop(['account'], axis = 1,inplace=True)
# df.drop(['caption'], axis = 1,inplace=True)

df.drop(['profile_name'], axis = 1,inplace=True)
df.drop(['biography'], axis = 1,inplace=True)

In [17]:
#calculating missing values in the dataset
def missing_Values(data):
  missing_values = data.isnull().sum()
  missing_per = (missing_values/data.shape[0])*100
  missing_table = pd.concat([missing_values,missing_per], axis=1, ignore_index=True) 
  missing_table.rename(columns={0:'Total Missing Values',1:'Missing %'}, inplace=True)
  return missing_table

missing_Values(df)

,Total Missing Values,Missing %
image_type,0,0.000000
account,0,0.000000
caption,330,0.372822
following,0,0.000000
likes,0,0.000000
posts_count,0,0.000000
followers,0,0.000000
datetime,0,0.000000
comments,0,0.000000
tags,0,0.000000


In [18]:
#fill the missing values with zero

df['likes'] = df['likes'].fillna(0)
df['biography_length'] = df['biography_length'].fillna(0)
df['profile_name_len'] = df['profile_name_len'].fillna(0)
df['caption_length'] = df['caption_length'].fillna(0)

In [19]:
#calculating missing values in the dataset
missing_Values(df)

,Total Missing Values,Missing %
image_type,0,0.000000
account,0,0.000000
caption,330,0.372822
following,0,0.000000
likes,0,0.000000
posts_count,0,0.000000
followers,0,0.000000
datetime,0,0.000000
comments,0,0.000000
tags,0,0.000000


## MetaFeatures

In [20]:
df1 = df[df['datetime'].notna()]

In [21]:
import datetime

def timeStampConversion(x):
    return datetime.datetime.fromtimestamp(int(x)/1000)

df1['datetime'] = pd.to_numeric(df1['datetime'], downcast="float")
df1['datetime'] = df1['datetime'].apply(lambda x: timeStampConversion(x)) 
df1["datetime"] = pd.to_datetime(df1["datetime"])

In [22]:
df1['Dow'] = df1['datetime'].dt.weekday
df1['Hod']= df1['datetime'].dt.hour
df1['Date']= df1['datetime'].dt.date

In [23]:
df1['mon']=(df1['Dow']==0).astype(int)
df1['tue']=(df1['Dow']==1).astype(int)
df1['wed']=(df1['Dow']==2).astype(int)
df1['thu']=(df1['Dow']==3).astype(int)
df1['fri']=(df1['Dow']==4).astype(int)  
df1['sat']=(df1['Dow']==5).astype(int)
df1['sun']=(df1['Dow']==6).astype(int)

In [24]:
df1

,image_type,account,caption,following,likes,posts_count,followers,datetime,comments,tags,...,Dow,Hod,Date,mon,tue,wed,thu,fri,sat,sun
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,352.0,29.0,280.0,8702.0,2021-07-18 09:19:41.056,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...",...,6,9,2021-07-18,0,0,0,0,0,0,1
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,352.0,44.0,280.0,8702.0,2022-03-10 13:45:42.144,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...",...,3,13,2022-03-10,0,0,0,1,0,0,0
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,352.0,209.0,280.0,8702.0,2021-10-20 11:39:44.000,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...",...,2,11,2021-10-20,0,0,1,0,0,0,0
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",352.0,95.0,280.0,8702.0,2021-10-01 12:06:51.136,3.0,"['food', 'bread', 'fast food', 'bun', 'america...",...,4,12,2021-10-01,0,0,0,0,1,0,0
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,352.0,101.0,280.0,8702.0,2021-07-22 05:00:20.352,6.0,"['text', 'font', 'design', 'yellow', 'graphics...",...,3,5,2021-07-22,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,579.0,10.0,1863.0,2321.0,2021-11-12 09:39:17.632,1.0,"['text', 'book', 'screenshot', 'brochure', 'pr...",...,4,9,2021-11-12,0,0,0,0,1,0,0
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,579.0,50.0,1863.0,2321.0,2021-11-11 08:40:42.240,0.0,"['building', 'text', 'electronic signage', 'ch...",...,3,8,2021-11-11,0,0,0,1,0,0,0
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,579.0,8.0,1863.0,2321.0,2021-11-10 13:14:09.792,0.0,"['text', 'food', 'snack']",...,2,13,2021-11-10,0,0,1,0,0,0,0
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,579.0,9.0,1863.0,2321.0,2021-11-10 13:14:09.792,0.0,"['text', 'poster', 'cartoon']",...,2,13,2021-11-10,0,0,1,0,0,0,0


In [25]:
df = pd.get_dummies(df1, columns=['industry'])

In [26]:
df

,image_type,account,caption,following,likes,posts_count,followers,datetime,comments,tags,...,sat,sun,industry_childcare,industry_cosmetics,industry_fashion,industry_fitness,industry_hospitality,industry_real estate,industry_retail,industry_sport & recreation
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,352.0,29.0,280.0,8702.0,2021-07-18 09:19:41.056,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...",...,0,1,0,0,0,0,1,0,0,0
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,352.0,44.0,280.0,8702.0,2022-03-10 13:45:42.144,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...",...,0,0,0,0,0,0,1,0,0,0
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,352.0,209.0,280.0,8702.0,2021-10-20 11:39:44.000,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...",...,0,0,0,0,0,0,1,0,0,0
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",352.0,95.0,280.0,8702.0,2021-10-01 12:06:51.136,3.0,"['food', 'bread', 'fast food', 'bun', 'america...",...,0,0,0,0,0,0,1,0,0,0
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,352.0,101.0,280.0,8702.0,2021-07-22 05:00:20.352,6.0,"['text', 'font', 'design', 'yellow', 'graphics...",...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,579.0,10.0,1863.0,2321.0,2021-11-12 09:39:17.632,1.0,"['text', 'book', 'screenshot', 'brochure', 'pr...",...,0,0,0,0,0,0,0,0,1,0
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,579.0,50.0,1863.0,2321.0,2021-11-11 08:40:42.240,0.0,"['building', 'text', 'electronic signage', 'ch...",...,0,0,0,0,0,0,0,0,1,0
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,579.0,8.0,1863.0,2321.0,2021-11-10 13:14:09.792,0.0,"['text', 'food', 'snack']",...,0,0,0,0,0,0,0,0,1,0
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,579.0,9.0,1863.0,2321.0,2021-11-10 13:14:09.792,0.0,"['text', 'poster', 'cartoon']",...,0,0,0,0,0,0,0,0,1,0


In [27]:
dropped = ['industry_fashion','industry_retail','industry_childcare','industry_fitness','industry_sport & recreation','industry_childcare','industry_real estate','Date','Dow','datetime']
df.drop(dropped,axis=1,inplace=True)
df

,image_type,account,caption,following,likes,posts_count,followers,comments,tags,confidence_score,...,Hod,mon,tue,wed,thu,fri,sat,sun,industry_cosmetics,industry_hospitality
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,352.0,29.0,280.0,8702.0,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...",...,9,0,0,0,0,0,0,1,0,1
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,352.0,44.0,280.0,8702.0,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...",...,13,0,0,0,1,0,0,0,0,1
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,352.0,209.0,280.0,8702.0,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...",...,11,0,0,1,0,0,0,0,0,1
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",352.0,95.0,280.0,8702.0,3.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...",...,12,0,0,0,0,1,0,0,0,1
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,352.0,101.0,280.0,8702.0,6.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...",...,5,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,579.0,10.0,1863.0,2321.0,1.0,"['text', 'book', 'screenshot', 'brochure', 'pr...","[0.9999996423721313, 0.9184167385101318, 0.910...",...,9,0,0,0,0,1,0,0,0,0
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,579.0,50.0,1863.0,2321.0,0.0,"['building', 'text', 'electronic signage', 'ch...","[0.9849988222122192, 0.9520490169525146, 0.921...",...,8,0,0,0,1,0,0,0,0,0
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,579.0,8.0,1863.0,2321.0,0.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...",...,13,0,0,1,0,0,0,0,0,0
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,579.0,9.0,1863.0,2321.0,0.0,"['text', 'poster', 'cartoon']","[0.9999863505363464, 0.9184243679046631, 0.910...",...,13,0,0,1,0,0,0,0,0,0


## NLP

In [28]:
data=df.astype(str)
def sentiment_analysis(caption):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity
  
    #Create a function to get the polarity
    def getPolarity(caption):
        return TextBlob(caption).sentiment.polarity
  
    #Create two new columns 'Subjectivity' & 'Polarity'
    data['TextBlob_Subjectivity'] = data['caption'].apply(getSubjectivity)
    data ['TextBlob_Polarity'] =  data['caption'].apply(getPolarity)
    return df

sentiment_analysis(data['caption'])

,image_type,account,caption,following,likes,posts_count,followers,comments,tags,confidence_score,...,Hod,mon,tue,wed,thu,fri,sat,sun,industry_cosmetics,industry_hospitality
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,352.0,29.0,280.0,8702.0,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...",...,9,0,0,0,0,0,0,1,0,1
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,352.0,44.0,280.0,8702.0,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...",...,13,0,0,0,1,0,0,0,0,1
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,352.0,209.0,280.0,8702.0,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...",...,11,0,0,1,0,0,0,0,0,1
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",352.0,95.0,280.0,8702.0,3.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...",...,12,0,0,0,0,1,0,0,0,1
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,352.0,101.0,280.0,8702.0,6.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...",...,5,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day 2 Christmas brochure tour Day one brochure...,579.0,10.0,1863.0,2321.0,1.0,"['text', 'book', 'screenshot', 'brochure', 'pr...","[0.9999996423721313, 0.9184167385101318, 0.910...",...,9,0,0,0,0,1,0,0,0,0
88510,"['Not Clip', 'Not LineDraw']",sandysfishmongers,#ArmisticeDay #poppy #lestwerespect,579.0,50.0,1863.0,2321.0,0.0,"['building', 'text', 'electronic signage', 'ch...","[0.9849988222122192, 0.9520490169525146, 0.921...",...,8,0,0,0,1,0,0,0,0,0
88511,"['Not Clip', 'Not LineDraw']",sandysfishmongers,There’s only one turkey and the best place to ...,579.0,8.0,1863.0,2321.0,0.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...",...,13,0,0,1,0,0,0,0,0,0
88512,"['Not Clip', 'Not LineDraw']",sandysfishmongers,Day one brochure tour !! #sandysfishmongers #b...,579.0,9.0,1863.0,2321.0,0.0,"['text', 'poster', 'cartoon']","[0.9999863505363464, 0.9184243679046631, 0.910...",...,13,0,0,1,0,0,0,0,0,0


In [29]:
hashtag_summary = adv.extract_hashtags(data['caption'])
emoji_summary = adv.extract_emoji(data['caption'])
mention_summary = adv.extract_mentions(data['caption'])

extracted =  (data.assign(hashtags=hashtag_summary['hashtags'],
         hashcounts=hashtag_summary['hashtag_counts'],
         mentions=mention_summary['mentions'],
         mention_count=mention_summary['mention_counts'],
        emoji=emoji_summary['emoji'],
        emoji_text=emoji_summary['emoji_text'],
        emoji_count=emoji_summary['emoji_counts'] 
         ))

extracted.head(5)

,image_type,account,caption,following,likes,posts_count,followers,comments,tags,confidence_score,...,industry_hospitality,TextBlob_Subjectivity,TextBlob_Polarity,hashtags,hashcounts,mentions,mention_count,emoji,emoji_text,emoji_count
0,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,It’s our last Pizza Sunday at the arch today. ...,352.0,29.0,280.0,8702.0,0.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...",...,1,0.233333,0.200000,[],0,[],0,"[🍕, 🍕, 🍕, 🥐, 🥐, 🥐]","[pizza, pizza, pizza, croissant, croissant, cr...",6
1,"['Not Clip', 'Not LineDraw']",yeastbakery,New cakes on our counters 😋😋,352.0,44.0,280.0,8702.0,0.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...",...,1,0.454545,0.136364,[],0,[],0,"[😋, 😋]","[face savoring food, face savoring food]",2
2,"['Not Clip', 'Not LineDraw']",yeastbakery,🔺Opening Times for Half Term🔺: Hi everyone we ...,352.0,209.0,280.0,8702.0,9.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...",...,1,0.147222,-0.080556,[],0,[],0,"[🔺, 🔺]","[red triangle pointed up, red triangle pointed...",2
3,"['Not Clip', 'Not LineDraw']",yeastbakery,"New addition to our selection of sandwiches, o...",352.0,95.0,280.0,8702.0,3.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...",...,1,0.477273,0.085227,[],0,[],0,[],[],0
4,"['Ambiguous Clip', 'Not LineDraw']",yeastbakery,We are currently closed as we move into our ne...,352.0,101.0,280.0,8702.0,6.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...",...,1,0.444719,0.174416,"[#yeastbakery, #eastlondon, #eastlondonfood, #...",4,[],0,[],[],0


In [34]:
##Hashtag popularity

def captionLength(i):
    return len(i)

caption_length = extracted['caption'].apply(captionLength)
extracted['caption_length'] = caption_length


#Converting the columns into the float datatype
a=['followers']
for i in a:
  extracted[i]= extracted[i].astype(str).astype(float)
extracted



word_freq_hash = adv.word_frequency(extracted['hashtags'].str.join(' '), extracted['followers'].fillna(0))
word_freq_hash.head()
# type(word_freq_hash.word)

d=pd.Series(word_freq_hash.rel_value.values,index=word_freq_hash.word.values ).to_dict()
print(d.get('#chelseabridgeclinic'))
print(d.get('#london'))
print(d.get('#test'))


def countHashtags(tags):
      if len(tags)==0:
        return 0
      sum = 0
      for i in tags:
        sum += (d.get(i))
      return sum/len(tags)

hashtag_average = extracted['hashtags'].apply(countHashtags)
extracted['hashtag_popularity'] = hashtag_average

None
None
None


TypeError: ignored

In [31]:
REPLACE_mentions = re.compile('@[A-Za-z0-9_]+')
REPLACE_hashtags=re.compile('#[A-Za-z0-9_]+')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

def lower(text: str) -> str:
    """
    Transforms given text to lower case.
    Example:
    Input: 'I really like New York city'
    Output: 'i really like new your city'
    """

    return text.lower()

def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)

def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace('br', '')

def filter_out_uncommon_symbols(text: str) -> str:
    """
    Removes any special character that is not in the
    good symbols list (check regular expression)
    """

    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text: str) -> str:
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])


def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()

def remove_brackets_from_list(text:str)->str:
    """
    Remove the [] brackets from the each of the list
    """
    return text

# def translation(text: str) -> str:
  
#     """
#     Replaces any non english words to the english language
#     """
#     translator = Translator()
#     res=translator.translate(text, dest='en')
#     return res.text


def clean_mention(text:str)-> str:
    return REPLACE_mentions.sub(' ', text)

def clean_hashtags(text:str)-> str:
    return REPLACE_hashtags.sub(' ', text)

In [32]:
PREPROCESSING_PIPELINE = [
                          clean_mention,
                          clean_hashtags,
                          # translation
                          lower,
                          replace_special_characters,
                          replace_br,
                          filter_out_uncommon_symbols,
                          remove_stopwords,
                          strip_text
                          ]

PREPROCESSING_PIPELINE1 = [remove_brackets_from_list]

# Anchor method

def text_prepare1(text: str,filter_methods = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE1
    return reduce(lambda txt, f: f(txt), filter_methods, text)

def text_prepare(text: str, filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing

print('Pre-processing text...')
print('[Debug] Before:\n{}'.format(extracted.caption[:10]))

L=['hashtags','emoji_text','mentions']
# L=['hashtags','mentions']
A=['caption']

# Replace each sentence with its pre-processed version
extracted['caption'] = extracted['caption'].apply(lambda txt: text_prepare(txt))

for i in L:
  extracted[i]=extracted[i].apply(lambda txt: text_prepare1(str(txt)[1:-1]))

print('[Debug] After:\n{}'.format(extracted.caption[:10]))
print()

print("Pre-processing completed!")


Pre-processing text...
[Debug] Before:
0    It’s our last Pizza Sunday at the arch today. ...
1                         New cakes on our counters 😋😋
2    🔺Opening Times for Half Term🔺: Hi everyone we ...
3    New addition to our selection of sandwiches, o...
4    We are currently closed as we move into our ne...
5    Weekend treats…\nIncluding the return of our “...
6    New vegan “Chocolate & Toasted Almond Cake wit...
7    Posted @withregram • @londoncoffeeshops Pastri...
8    We are looking for experienced part-time baris...
9    Posted @withregram • @marthadelacey Thrilled f...
Name: caption, dtype: object
[Debug] After:
0    last pizza sunday arch today pizzas available ...
1                                   new cakes counters
2    opening times half term hi everyone wanted let...
3    new addition selection sandwiches roasted aube...
4    currently closed move new exciting space aroun...
5    weekend treatsincluding return coconut custard...
6    new vegan chocolate toasted almo

In [35]:
# Hashtags

extracted['hashtag_max'] = extracted.groupby('account')['hashcounts'].transform('max')
extracted['hashtag_min'] = extracted.groupby('account')['hashcounts'].transform('min')
extracted['hashtag_mean'] = extracted.groupby('account')['hashcounts'].transform('mean')
extracted['hashtag_std'] = extracted.groupby('account')['hashcounts'].transform('std')


In [82]:
extracted.drop('account',1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [40]:
REPLACE_BY_SPACE_RE = re.compile('[^a-zA-Z_0-9,.]')

def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()


def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace("'", "")


def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)


from typing import List, Callable, Dict

PREPROCESSING_PIPELINE = [
                          replace_special_characters,
                          strip_text
                          ]



def text_prepare(text: str, filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing

print('Pre-processing text...')
print('[Debug] Before:\n{}'.format(extracted.tags[:10]))

L=['tags','confidence_score','accent_color','dominant_colors','bg_color','fore_color']

A=['tags']

for i in L:
  extracted[i]=extracted[i].apply(lambda txt: text_prepare(str(txt)[1:-1]))

print('[Debug] After:\n{}'.format(extracted.tags[:10]))
print()

print("Pre-processing completed!")

Pre-processing text...
[Debug] Before:
0    ['text', 'font', 'graphics', 'screenshot', 'gr...
1    ['dessert', 'baked goods', 'baking', 'snack', ...
2    ['food', 'fruit', 'baked goods', 'breakfast', ...
3    ['food', 'bread', 'fast food', 'bun', 'america...
4    ['text', 'font', 'design', 'yellow', 'graphics...
5    ['snack', 'baked goods', 'baking', 'dessert', ...
6    ['dessert', 'baked goods', 'baking', 'sweetnes...
7    ['text', 'food', 'shelf', 'countertop', 'indoo...
8    ['text', 'table', 'food', 'bakery', 'clothing'...
9    ['baked goods', 'sweetness', 'food', 'baking',...
Name: tags, dtype: object
[Debug] After:
0    text ,  font ,  graphics ,  screenshot ,  grap...
1    dessert ,  baked goods ,  baking ,  snack ,  f...
2    food ,  fruit ,  baked goods ,  breakfast ,  c...
3    food ,  bread ,  fast food ,  bun ,  american ...
4    text ,  font ,  design ,  yellow ,  graphics ,...
5    snack ,  baked goods ,  baking ,  dessert ,  b...
6    dessert ,  baked goods ,  baking , 

In [41]:
extracted['image_type']=extracted['image_type'].apply(lambda txt: text_prepare(str(txt)[1:-1]))

In [42]:
df = pd.get_dummies(extracted, columns=['image_type'])

In [43]:
df

,account,caption,following,likes,posts_count,followers,comments,tags,confidence_score,accent_color,...,hashtag_mean,hashtag_std,"image_type_Ambiguous Clip , LineDraw","image_type_Ambiguous Clip , Not LineDraw","image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw"
0,yeastbakery,last pizza sunday arch today pizzas available ...,352.0,29.0,280.0,8702.0,0.0,"text , font , graphics , screenshot , grap...","0.9980798959732056, 0.9481294751167297, 0.8818...","0.788235294117647, 0.00784313725490196, 0.0078...",...,0.833333,1.508919,0,1,0,0,0,0,0,0
1,yeastbakery,new cakes counters,352.0,44.0,280.0,8702.0,0.0,"dessert , baked goods , baking , snack , f...","0.9897554516792297, 0.987897515296936, 0.98287...","0.6901960784313725, 0.14901960784313725, 0.105...",...,0.833333,1.508919,0,0,0,0,0,0,0,1
2,yeastbakery,opening times half term hi everyone wanted let...,352.0,209.0,280.0,8702.0,9.0,"food , fruit , baked goods , breakfast , c...","0.9808361530303955, 0.9546540379524231, 0.9477...","0.6588235294117647, 0.4392156862745098, 0.1411...",...,0.833333,1.508919,0,0,0,0,0,0,0,1
3,yeastbakery,new addition selection sandwiches roasted aube...,352.0,95.0,280.0,8702.0,3.0,"food , bread , fast food , bun , american ...","0.9958561658859253, 0.981575608253479, 0.97992...","0.24313725490196078, 0.12549019607843137, 0.07...",...,0.833333,1.508919,0,0,0,0,0,0,0,1
4,yeastbakery,currently closed move new exciting space aroun...,352.0,101.0,280.0,8702.0,6.0,"text , font , design , yellow , graphics ,...","0.9993002414703369, 0.9640201926231384, 0.9513...","0.792156862745098, 0.7372549019607844, 0.00392...",...,0.833333,1.508919,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,sandysfishmongers,day 2 christmas ochure tour day one ochure tour #,579.0,10.0,1863.0,2321.0,1.0,"text , book , screenshot , brochure , prin...","0.9999996423721313, 0.9184167385101318, 0.9103...","0.011764705882352941, 0.5019607843137255, 0.78...",...,10.144144,7.473404,0,0,0,0,0,0,0,1
88510,sandysfishmongers,,579.0,50.0,1863.0,2321.0,0.0,"building , text , electronic signage , chri...","0.9849988222122192, 0.9520490169525146, 0.9216...","0.6470588235294118, 0.1568627450980392, 0.1490...",...,10.144144,7.473404,0,0,0,0,0,0,0,1
88511,sandysfishmongers,theres one turkey best place buy sandys make c...,579.0,8.0,1863.0,2321.0,0.0,"text , food , snack","0.9999812841415405, 0.9580191373825073, 0.9157...","0.6588235294117647, 0.00784313725490196, 0.713...",...,10.144144,7.473404,0,0,0,0,0,0,0,1
88512,sandysfishmongers,day one ochure tour #,579.0,9.0,1863.0,2321.0,0.0,"text , poster , cartoon","0.9999863505363464, 0.9184243679046631, 0.9104...","0.01568627450980392, 0.5098039215686274, 0.780...",...,10.144144,7.473404,0,0,0,0,0,0,0,1


In [44]:
# for i in df1['confidence_score']:

def convert_str2int(i):
  n = []
  y = i.split(', ')
  if len(y[0]) == 0:
    return []
  for j in range(len(y)):
    n.append(float(y[j]))
  return n

df['confidence_score'] = df['confidence_score'].apply(convert_str2int)
df['accent_color'] = df['accent_color'].apply(convert_str2int)
df['dominant_colors'] = df['dominant_colors'].apply(convert_str2int)
df['bg_color'] = df['bg_color'].apply(convert_str2int)
df['fore_color'] = df['fore_color'].apply(convert_str2int)

In [45]:
df

,account,caption,following,likes,posts_count,followers,comments,tags,confidence_score,accent_color,...,hashtag_mean,hashtag_std,"image_type_Ambiguous Clip , LineDraw","image_type_Ambiguous Clip , Not LineDraw","image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw"
0,yeastbakery,last pizza sunday arch today pizzas available ...,352.0,29.0,280.0,8702.0,0.0,"text , font , graphics , screenshot , grap...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",...,0.833333,1.508919,0,1,0,0,0,0,0,0
1,yeastbakery,new cakes counters,352.0,44.0,280.0,8702.0,0.0,"dessert , baked goods , baking , snack , f...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",...,0.833333,1.508919,0,0,0,0,0,0,0,1
2,yeastbakery,opening times half term hi everyone wanted let...,352.0,209.0,280.0,8702.0,9.0,"food , fruit , baked goods , breakfast , c...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",...,0.833333,1.508919,0,0,0,0,0,0,0,1
3,yeastbakery,new addition selection sandwiches roasted aube...,352.0,95.0,280.0,8702.0,3.0,"food , bread , fast food , bun , american ...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",...,0.833333,1.508919,0,0,0,0,0,0,0,1
4,yeastbakery,currently closed move new exciting space aroun...,352.0,101.0,280.0,8702.0,6.0,"text , font , design , yellow , graphics ,...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",...,0.833333,1.508919,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,sandysfishmongers,day 2 christmas ochure tour day one ochure tour #,579.0,10.0,1863.0,2321.0,1.0,"text , book , screenshot , brochure , prin...","[0.9999996423721313, 0.9184167385101318, 0.910...","[0.011764705882352941, 0.5019607843137255, 0.7...",...,10.144144,7.473404,0,0,0,0,0,0,0,1
88510,sandysfishmongers,,579.0,50.0,1863.0,2321.0,0.0,"building , text , electronic signage , chri...","[0.9849988222122192, 0.9520490169525146, 0.921...","[0.6470588235294118, 0.1568627450980392, 0.149...",...,10.144144,7.473404,0,0,0,0,0,0,0,1
88511,sandysfishmongers,theres one turkey best place buy sandys make c...,579.0,8.0,1863.0,2321.0,0.0,"text , food , snack","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.00784313725490196, 0.71...",...,10.144144,7.473404,0,0,0,0,0,0,0,1
88512,sandysfishmongers,day one ochure tour #,579.0,9.0,1863.0,2321.0,0.0,"text , poster , cartoon","[0.9999863505363464, 0.9184243679046631, 0.910...","[0.01568627450980392, 0.5098039215686274, 0.78...",...,10.144144,7.473404,0,0,0,0,0,0,0,1


In [46]:
def convert_str2list(i):
  n = []
  y = i.split(', ')
  for j in range(len(y)):
    n.append(str(y[j]))
  return n
df['tags'] = df['tags'].apply(convert_str2list)
df['tags'][0]

['text ',
 ' font ',
 ' graphics ',
 ' screenshot ',
 ' graphic design ',
 ' design ',
 ' typography']

In [47]:
df['confidence_score'][6]

[0.9938297271728516,
 0.9784528613090515,
 0.977806806564331,
 0.966992974281311,
 0.9616279602050781,
 0.9515609741210938,
 0.9280351400375366,
 0.912075936794281,
 0.9083282947540283,
 0.9079588651657104,
 0.8951475620269775,
 0.8874362707138062,
 0.8836393356323242,
 0.8746436834335327,
 0.8713182210922241,
 0.8668385744094849,
 0.8426526784896851,
 0.786042332649231,
 0.5681669116020203]

In [48]:
def trim_lowConfidence_tags(tag_column):
  for scoreList in df['confidence_score']:
    for score_index in range(len(scoreList)):
      if scoreList[score_index]<0.5:
        return tag_column[:score_index]

df['tags'] = df['tags'].apply(trim_lowConfidence_tags)  

In [49]:
df.drop('confidence_score',1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [50]:
df.dominant_colors[1]

[0.6470588235294118, 0.16470588235294117, 0.16470588235294117]

In [51]:
df['dominant_colors'][:3]

0         [1.0, 0.7529411764705882, 0.796078431372549]
1    [0.6470588235294118, 0.16470588235294117, 0.16...
2    [0.6470588235294118, 0.16470588235294117, 0.16...
Name: dominant_colors, dtype: object

In [52]:
def separate_column(column):
  if len(column)>0:
    return column[0]
  else:
    return []

df['dominant_colors_R'] = df['dominant_colors'].apply(separate_column)  
df['accent_color_R'] = df['accent_color'].apply(separate_column)  
df['bg_color_R'] = df['bg_color'].apply(separate_column)  
df['fore_color_R'] = df['fore_color'].apply(separate_column)  

In [53]:
df

,account,caption,following,likes,posts_count,followers,comments,tags,accent_color,is_bw,...,"image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw",dominant_colors_R,accent_color_R,bg_color_R,fore_color_R
0,yeastbakery,last pizza sunday arch today pizzas available ...,352.0,29.0,280.0,8702.0,0.0,"[text , font , graphics , screenshot , gra...","[0.788235294117647, 0.00784313725490196, 0.007...",0,...,0,0,0,0,0,0,1.000000,0.788235,1.000000,1.000000
1,yeastbakery,new cakes counters,352.0,44.0,280.0,8702.0,0.0,"[dessert , baked goods , baking , snack , ...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,...,0,0,0,0,0,1,0.647059,0.690196,0.647059,0.000000
2,yeastbakery,opening times half term hi everyone wanted let...,352.0,209.0,280.0,8702.0,9.0,"[food , fruit , baked goods , breakfast , ...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,...,0,0,0,0,0,1,0.647059,0.658824,0.647059,0.647059
3,yeastbakery,new addition selection sandwiches roasted aube...,352.0,95.0,280.0,8702.0,3.0,"[food , bread , fast food , bun , american...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,...,0,0,0,0,0,1,0.647059,0.243137,0.647059,0.647059
4,yeastbakery,currently closed move new exciting space aroun...,352.0,101.0,280.0,8702.0,6.0,"[text , font , design , yellow , graphics ...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,...,0,0,0,0,0,0,1.000000,0.792157,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,sandysfishmongers,day 2 christmas ochure tour day one ochure tour #,579.0,10.0,1863.0,2321.0,1.0,"[text , book , screenshot , brochure , pri...","[0.011764705882352941, 0.5019607843137255, 0.7...",0,...,0,0,0,0,0,1,1.000000,0.011765,0.000000,1.000000
88510,sandysfishmongers,,579.0,50.0,1863.0,2321.0,0.0,"[building , text , electronic signage , chr...","[0.6470588235294118, 0.1568627450980392, 0.149...",0,...,0,0,0,0,0,1,0.501961,0.647059,0.000000,1.000000
88511,sandysfishmongers,theres one turkey best place buy sandys make c...,579.0,8.0,1863.0,2321.0,0.0,"[text , food , snack]","[0.6588235294117647, 0.00784313725490196, 0.71...",0,...,0,0,0,0,0,1,0.501961,0.658824,0.501961,0.501961
88512,sandysfishmongers,day one ochure tour #,579.0,9.0,1863.0,2321.0,0.0,"[text , poster , cartoon]","[0.01568627450980392, 0.5098039215686274, 0.78...",0,...,0,0,0,0,0,1,1.000000,0.015686,1.000000,1.000000


In [ ]:
# def separate_column(column):
#   if len(column)>0:
#     if column[1]=='':
#       return 'NAN'
#     else:
#       return column[1]
#   else:
#     return []

# df['dominant_colors_G'] = df['dominant_colors'].apply(separate_column)  
# df['accent_color_G'] = df['accent_color'].apply(separate_column)  
# df['bg_color_G'] = df['bg_color'].apply(separate_column)  
# df['fore_color_G'] = df['fore_color'].apply(separate_column)  

In [ ]:
# def separate_column(column):
#   if len(column)>0:
#     return column[2]
#   else:
#     return []

# df['dominant_colors_B'] = df['dominant_colors'].apply(separate_column)  
# df['accent_color_B'] = df['accent_color'].apply(separate_column)  
# df['bg_color_B'] = df['bg_color'].apply(separate_column)  
# df['fore_color_B'] = df['fore_color'].apply(separate_column)  

In [ ]:
# df['image_type_A'] = df['image_type'].apply(separate_column)  

In [ ]:
# df['image_type_B'] = df['image_type'].apply(separate_column)  

In [ ]:
# df

In [54]:
dropped = ['accent_color', 'dominant_colors', 'bg_color','fore_color']
data = df.drop(dropped,axis=1)

In [55]:
data

,account,caption,following,likes,posts_count,followers,comments,tags,is_bw,caption_length,...,"image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw",dominant_colors_R,accent_color_R,bg_color_R,fore_color_R
0,yeastbakery,last pizza sunday arch today pizzas available ...,352.0,29.0,280.0,8702.0,0.0,"[text , font , graphics , screenshot , gra...",0,72,...,0,0,0,0,0,0,1.000000,0.788235,1.000000,1.000000
1,yeastbakery,new cakes counters,352.0,44.0,280.0,8702.0,0.0,"[dessert , baked goods , baking , snack , ...",0,18,...,0,0,0,0,0,1,0.647059,0.690196,0.647059,0.000000
2,yeastbakery,opening times half term hi everyone wanted let...,352.0,209.0,280.0,8702.0,9.0,"[food , fruit , baked goods , breakfast , ...",0,178,...,0,0,0,0,0,1,0.647059,0.658824,0.647059,0.647059
3,yeastbakery,new addition selection sandwiches roasted aube...,352.0,95.0,280.0,8702.0,3.0,"[food , bread , fast food , bun , american...",0,104,...,0,0,0,0,0,1,0.647059,0.243137,0.647059,0.647059
4,yeastbakery,currently closed move new exciting space aroun...,352.0,101.0,280.0,8702.0,6.0,"[text , font , design , yellow , graphics ...",0,121,...,0,0,0,0,0,0,1.000000,0.792157,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,sandysfishmongers,day 2 christmas ochure tour day one ochure tour #,579.0,10.0,1863.0,2321.0,1.0,"[text , book , screenshot , brochure , pri...",0,49,...,0,0,0,0,0,1,1.000000,0.011765,0.000000,1.000000
88510,sandysfishmongers,,579.0,50.0,1863.0,2321.0,0.0,"[building , text , electronic signage , chr...",0,0,...,0,0,0,0,0,1,0.501961,0.647059,0.000000,1.000000
88511,sandysfishmongers,theres one turkey best place buy sandys make c...,579.0,8.0,1863.0,2321.0,0.0,"[text , food , snack]",0,110,...,0,0,0,0,0,1,0.501961,0.658824,0.501961,0.501961
88512,sandysfishmongers,day one ochure tour #,579.0,9.0,1863.0,2321.0,0.0,"[text , poster , cartoon]",0,21,...,0,0,0,0,0,1,1.000000,0.015686,1.000000,1.000000


In [58]:
d2=data

In [59]:
data2=data.drop('tags',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [60]:
data3=data.drop('caption',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [61]:
data33=data3.astype('object')
data22=data2.astype('object')

In [62]:
type(data2)

pandas.core.frame.DataFrame

In [63]:
d2['tags']=d2['tags'].apply(lambda txt: text_prepare(str(txt)[1:-1]))
tokenized_tag = d2['tags'].apply(lambda x: x.split()[1:-1])
tokenized_tag[1]

[',',
 'baked',
 'goods',
 ',',
 'baking',
 ',',
 'snack',
 ',',
 'food',
 ',',
 'sweetness',
 ',',
 'finger',
 'food',
 ',',
 'bakery',
 ',',
 'petit',
 'four',
 ',',
 'muffin',
 ',',
 'indoor',
 ',',
 'kue',
 ',']

##FastText

In [64]:
modelFastTextSkipGram = FastText(tokenized_tag, 
                     size=50, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGram.build_vocab(tokenized_tag, update=True)
modelFastTextSkipGram.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=2)

2022-05-14 19:09:22,529 | WARNING | base_any2vec.py:723 | __init__ | consider setting layer size to a multiple of 4 for greater performance
2022-05-14 19:09:22,532 | INFO | word2vec.py:1567 | scan_vocab | collecting all words and their counts
2022-05-14 19:09:22,535 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-14 19:09:22,576 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #10000, processed 184916 words, keeping 2209 word types
2022-05-14 19:09:22,619 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #20000, processed 375164 words, keeping 2671 word types
2022-05-14 19:09:22,657 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #30000, processed 562939 words, keeping 2938 word types
2022-05-14 19:09:22,697 | INFO | word2vec.py:1552 | _scan_vocab | PROGRESS: at sentence #40000, processed 754746 words, keeping 3128 word types
2022-05-14 19:09:22,739 | INFO | word2vec.py:1552

In [ ]:
modelFastTextSkipGram.wv.most_similar("Gastroenteritis")

In [ ]:
modelFastTextSkipGram.wv.most_similar("pizza")

In [67]:
def word_vector(tokens, size,model):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [68]:
fastText_arrays = np.zeros((len(tokenized_tag), 50)) 
for i in range(len(tokenized_tag)):
    fastText_arrays[i,:] = word_vector(tokenized_tag[i], 50,modelFastTextSkipGram)
fastText_df = pd.DataFrame(fastText_arrays)
fastText_df.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(88514, 50)

In [69]:
type(data2)

pandas.core.frame.DataFrame

In [70]:
data22

,account,caption,following,likes,posts_count,followers,comments,is_bw,caption_length,biography_length,...,"image_type_Good Clip , LineDraw","image_type_Good Clip , Not LineDraw","image_type_Normal Clip , LineDraw","image_type_Normal Clip , Not LineDraw","image_type_Not Clip , LineDraw","image_type_Not Clip , Not LineDraw",dominant_colors_R,accent_color_R,bg_color_R,fore_color_R
0,yeastbakery,last pizza sunday arch today pizzas available ...,352.0,29.0,280.0,8702.0,0.0,0,72,149.0,...,0,0,0,0,0,0,1.0,0.788235,1.0,1.0
1,yeastbakery,new cakes counters,352.0,44.0,280.0,8702.0,0.0,0,18,149.0,...,0,0,0,0,0,1,0.647059,0.690196,0.647059,0.0
2,yeastbakery,opening times half term hi everyone wanted let...,352.0,209.0,280.0,8702.0,9.0,0,178,149.0,...,0,0,0,0,0,1,0.647059,0.658824,0.647059,0.647059
3,yeastbakery,new addition selection sandwiches roasted aube...,352.0,95.0,280.0,8702.0,3.0,0,104,149.0,...,0,0,0,0,0,1,0.647059,0.243137,0.647059,0.647059
4,yeastbakery,currently closed move new exciting space aroun...,352.0,101.0,280.0,8702.0,6.0,0,121,149.0,...,0,0,0,0,0,0,1.0,0.792157,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88509,sandysfishmongers,day 2 christmas ochure tour day one ochure tour #,579.0,10.0,1863.0,2321.0,1.0,0,49,150.0,...,0,0,0,0,0,1,1.0,0.011765,0.0,1.0
88510,sandysfishmongers,,579.0,50.0,1863.0,2321.0,0.0,0,0,150.0,...,0,0,0,0,0,1,0.501961,0.647059,0.0,1.0
88511,sandysfishmongers,theres one turkey best place buy sandys make c...,579.0,8.0,1863.0,2321.0,0.0,0,110,150.0,...,0,0,0,0,0,1,0.501961,0.658824,0.501961,0.501961
88512,sandysfishmongers,day one ochure tour #,579.0,9.0,1863.0,2321.0,0.0,0,21,150.0,...,0,0,0,0,0,1,1.0,0.015686,1.0,1.0


In [72]:
# # data22.drop(['image_type'], axis = 1,inplace=True)
# # data22.drop(['datetime'], axis = 1,inplace=True)
# data22.drop(['industry'], axis = 1,inplace=True)

In [78]:
# dropped = ['accent_color', 'dominant_colors', 'bg_color','fore_color']
# data22.drop(dropped,axis=1,inplace=True)

In [ ]:
data22

In [79]:
dataset_fastText2 = pd.DataFrame(np.hstack((fastText_df,data22)))

In [80]:
dataset_fastText2 = dataset_fastText2.reset_index()

In [83]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

clean_dataset(dataset_fastText2)

ValueError: ignored

## MinMaxScaling

In [ ]:
from sklearn import preprocessing
df = pd.DataFrame(dataset_fastText2)
dataset_fastText33=df.astype(str)

normalizer = preprocessing.MinMaxScaler(feature_range=(0, 1))
normalized_df = pd.DataFrame(normalizer.fit_transform(dataset_fastText33),  columns = dataset_fastText33.columns)

In [ ]:
normalized_df[51]

In [ ]:
y=normalized_df[51]
X1=normalized_df.drop(51,1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=2022)

### LightLGBM

In [ ]:
# hyperparameterTuning_XGBoost(x_train,y_train)

In [ ]:
from lightgbm import LGBMRegressor
model_lgb = LGBMRegressor(colsample_bytree=0.8,learning_rate=0.01, max_depth=8,min_child_weight=1, min_split_gain=0.0222415, n_estimators=35000,num_leaves=966, reg_alpha=0.04, reg_lambda=0.073,subsample=0.6)

In [ ]:
start = datetime.datetime.now()
lgbm = model_lgb.fit(x_train, y_train,eval_set=[(x_train, y_train), (x_test, y_test)],eval_metric ='mae')
end = datetime.datetime.now()
end-start

In [ ]:
y_pred_lgbm = lgbm.predict(x_test)
lgbm.score(x_test, y_test)

In [ ]:
import lightgbm as lgb
lgb.plot_metric(lgbm)

In [ ]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_lgbm)
print(R2)

In [ ]:
dict_info = {'Model': [], 'RMSE': [], 'R2': [],'MAE': [], 'RMSLE': [],'max_error_':[]}

rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_lgbm)

dict_info['Model'].append('Lightgbm')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

###XGBoost model

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
        enable_categorical=True,
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.05,
        max_depth = 6,
        min_child_weight = 1,
        n_estimators = 1000,
        subsample = 0.7)

xgb_model.fit(x_train, y_train,eval_set=[(x_train, y_train), (x_test, y_test)], eval_metric='mae')

In [ ]:
y_pred_W2vec_xgboost = xgb_model.predict(x_test)
xgb_model.score(x_test, y_test)

In [ ]:
xgb_model.score(x_test, y_test)

In [ ]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_W2vec_xgboost)

dict_info['Model'].append('XGboost')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

## Random Forest

In [ ]:
# utils.hyperparameterTuning_RandomForest(x_train_w2v_df,y_train_w2v_df)
rnd_clf = RandomForestRegressor(n_estimators=20, min_samples_split=10, min_samples_leaf=4,max_features='sqrt',max_depth=10,bootstrap=True)
rnd_clf.fit(x_train, y_train)
y_pred_w2vec_rnd = rnd_clf.predict(x_test)

In [ ]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_w2vec_rnd)
print(R2)

dict_info['Model'].append('Random Forest')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

## MLP

In [ ]:
from sklearn.neural_network import MLPRegressor
mlp_GS = MLPRegressor(activation='relu',alpha=0.001,hidden_layer_sizes=(50, 50, 50),learning_rate='constant',solver='adam')
mlp_GS.fit(x_train, y_train)
y_pred_w2vec_mlp = mlp_GS.predict(x_test)

In [ ]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_w2vec_mlp)

dict_info['Model'].append('MLP')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

## SVC

In [ ]:
from sklearn.svm import LinearSVR
svr= LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,tol=0.0001, verbose=0)
svr.fit(x_train, y_train)
y_pred_w2vec_SVR = svr.predict(x_test)

In [ ]:
rmse,R2,MAE,MSE,RMSLE,max_error_= metrics(y_test,y_pred_w2vec_SVR)

dict_info['Model'].append('SVR')
dict_info['RMSE'].append(rmse)
dict_info['R2'].append(R2)
dict_info['MAE'].append(MAE)
dict_info['RMSLE'].append(RMSLE)
dict_info['max_error_'].append(max_error_)

In [ ]:
dfData = pd.DataFrame(dict_info)
dfData